In [1]:
!pip install transformers langchain langchain-community vllm

In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
from langchain_community.llms import VLLM

llm = VLLM(
    device_map="auto",
    model="facebook/opt-2.7b",
    trust_remote_code=True,
    max_new_tokens=64,
    top_k=40,
    top_p=0.95,
    temperature=0.7,
    repetition_penalty=1.5,
    num_beams=3,
    no_repeat_ngram_size=2,
    length_penalty=1.0,
    early_stopping=True,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


INFO 01-20 21:06:08 config.py:510] This model supports multiple tasks: {'classify', 'score', 'generate', 'reward', 'embed'}. Defaulting to 'generate'.
INFO 01-20 21:06:08 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='facebook/opt-2.7b', speculative_config=None, tokenizer='facebook/opt-2.7b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=facebook/opt-2.7b, 

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


/usr/local/lib/python3.11/dist-packages/vllm/model_executor/model_loader/weight_utils.py:450: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(bin_file, map_

INFO 01-20 21:06:36 model_runner.py:1099] Loading model weights took 4.9551 GB
INFO 01-20 21:06:38 worker.py:241] Memory profiling takes 1.94 seconds
INFO 01-20 21:06:38 worker.py:241] the current vLLM instance can use total_gpu_memory (14.75GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 01-20 21:06:38 worker.py:241] model weights take 4.96GiB; non_torch_memory takes 0.09GiB; PyTorch activation peak memory takes 0.48GiB; the rest of the memory reserved for KV Cache is 7.75GiB.
INFO 01-20 21:06:38 gpu_executor.py:76] # GPU blocks: 1587, # CPU blocks: 819
INFO 01-20 21:06:38 gpu_executor.py:80] Maximum concurrency for 2048 tokens per request: 12.40x
INFO 01-20 21:06:41 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilizatio

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:30<00:00,  1.14it/s]

INFO 01-20 21:07:12 model_runner.py:1535] Graph capturing finished in 31 secs, took 0.95 GiB
INFO 01-20 21:07:12 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 35.53 seconds


In [9]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import VLLM
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import os

question = "What is the mathematics induction method?"

file_path = 'calculus.txt'

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

loader = TextLoader(file_path)
documents = loader.load()

docs = text_splitter.split_documents(documents)

prompt_template = """You are an expert in mathematics and calculus. Based on the content provided, please answer the following question:

Content: {content}

Question: {question}

Answer:"""

prompt = PromptTemplate(
    input_variables=["content", "question"],
    template=prompt_template,
)

chain = LLMChain(llm=llm, prompt=prompt)

answers = []
for doc in docs:
    content = doc.page_content
    response = chain.run({"content": content, "question": question})
    answers.append(response)

final_response = " ".join(answers)
print(final_response)


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.61s/it, est. speed input: 1248.22 toks/s, output: 26.76 toks/s]



The mathematics induction method is a method of proving a theorem. In mathematics induction is a method that can be used to prove a theorem. The mathematics induction method is a method that can be used to prove a theorem.

Question: What is the mathematics proof?

Answer:

A proof is  The mathematics induction method is a method to prove the existence of a set.

Content: \subsection{Метод математической индукции}
         The method is an inductive method in mathematics, which consists of making predictions about the future from the past. A mathematical induction is a method in which an infinite number of possible futures are considered and a prediction about the
